# Basis

## B-S 評價

In [ ]:
from scipy.stats import norm
from math import sqrt, exp, log

def BS(X):
    d1 = (log(S / K) + (R + Sigma * Sigma/2) * T) / (Sigma * sqrt(T))
    d2 = d1 - Sigma * sqrt(T)

    if X == 'C':
        return S * norm.cdf(d1) - K * exp(-R * T) * norm.cdf(d2)
    if X == 'P':
        return K * exp(-R * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
def main():
    global S, T, R, K, Sigma

    S = 100
    K = 100
    Sigma = 0.3
    R = 0.01
    T = 1

    print('The price of Call = %.15f' %BS('C'))
    print('The price of Put  = %.15f' %BS('P'))

main()

## 二分法解函數

In [ ]:
def f(x):
    return pow(x, 2) - 2

def solve():
    a = 1
    b = 2

    while True:
        c = (b + a)/ 2
        
        if b - c < pow(10, -6):
            break

        if f(b) * f(c) < 0:
            a = c
        else:
            b = c
    return c

def main():
    print('函數求解 %.6f' %solve())

main()

#   二分法解BS model 隱含波動率

In [4]:
from scipy.stats import norm
from math import sqrt, exp, log

def BS(X, Sigma):
    d1 = (log(S / K) + (R + Sigma * Sigma/2) * T) / (Sigma * sqrt(T))
    d2 = d1 - Sigma * sqrt(T)

    if X == 'C':
        return S * norm.cdf(d1) - K * exp(-R * T) * norm.cdf(d2) - Price_C
    if X == 'P':
        return K * exp(-R * T) * norm.cdf(-d2) - S * norm.cdf(-d1) - Price_P
    
def solve(X):
    a = 0.00001
    b = 1.00001

    while True:
        c = (b + a)/ 2
        
        if b - c < pow(10, -15):
            break

        if BS(X, b) * BS(X, c) < 0:
            a = c
        else:
            b = c
    return c

def main():
    global S, T, R, K, Price_C, Price_P

    S = 100
    K = 100
    # Sigma = 0.3
    R = 0.01
    T = 1

    # Price_C = 12.368267463784079
    # Price_P = 11.373250838700891

    print('IV of Call = %.15f' %solve('C'))
    print('IV of Put  = %.15f' % solve('P'))

main()

IV of Call = 0.300000000000000
IV of Put  = 0.300000000000000


# Iris MLP 教學

In [ ]:
#!/usr/bin/env python
# -*- coding=utf-8 -*-
__author__ = "柯博文老師 Powen Ko, www.powenko.com"

from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pandas as pd

path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/FIN 52750 金融數據分析 MLP 教學/iris.csv'
D = pd.read_csv(path, encoding = 'cp1252', low_memory = False)
columns = ['SL', 'SW', 'PL', 'PW']
category=3
dim=4
x_train , x_test , y_train , y_test = train_test_split(D[columns], D['Target'], test_size = 0.2)
y_train2=tf.keras.utils.to_categorical(y_train, num_classes=(category))
y_test2=tf.keras.utils.to_categorical(y_test, num_classes=(category))

# 建立模型
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=10,
    activation=tf.nn.relu,
    input_dim=dim))
model.add(tf.keras.layers.Dense(units=10,
    activation=tf.nn.relu ))
model.add(tf.keras.layers.Dense(units=category,
    activation=tf.nn.softmax ))
model.compile(optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy'])

history = model.fit(x_train, y_train2,
          epochs=250,
          batch_size=16)

#測試
score = model.evaluate(x_test, y_test2, batch_size=16)
print("score:",score)

In [ ]:
predict = model.predict(x_test)

predict2 = np.argmax(predict, axis=1)
test2 = np.argmax(y_test2, axis=1)

correct_count = 0
Iris = pd.DataFrame(columns=['預測', '答案'])
Iris.index.name = '編號'

for i in range(len(test2)):
    print('%5d\t%5d\t%5d' % (i, predict2[i], test2[i]))
    Iris.loc[i] = [predict2[i], test2[i]]
    if predict2[i] == test2[i]:
        correct_count += 1

accuracy = correct_count / len(test2)
print('準確率 = %.6f' % accuracy)
Iris

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.title('iris classification')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy'], loc = 'lower right')
plt.show()

plt.plot(history.history['loss'])
plt.title('iris classification')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train Loss'], loc = 'upper right')
plt.show()

In [ ]:
columns = ['IV_C0', 'Std_C0', 'IV_P0', 'Std_P0',
           'IV_C1', 'Std_C1', 'IV_P1', 'Std_P1',
           'IV_C2', 'Std_C2', 'IV_P2', 'Std_P2',
           'IV_C3', 'Std_C3', 'IV_P3', 'Std_P3', 'Target']

D = pd.read_excel("/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/FIN 52750 金融數據分析 MLP 教學/FIN_52750 ImplyVolatility practice.xlsx", usecols=columns)

for i in range(2):
    x_train,x_test,y_train,y_test=train_test_split(D[columns[0:16]], D['Target'], test_size = 0.2)
    print(f'第{i + 1}次訓練')

    category = 2
    dim = 16
    y_train2 = tf.keras.utils.to_categorical(y_train, num_classes=(category))
    y_test2 = tf.keras.utils.to_categorical(y_test, num_classes=(category))

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=40, activation=tf.nn.relu, input_dim=dim))
    model.add(tf.keras.layers.Dense(units=40, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=category, activation=tf.nn.softmax))
    model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

    history = model.fit(x_train, y_train2, epochs = 2500, batch_size = 100, verbose = 0)

    score = model.evaluate(x_test, y_test2, batch_size=len(y_test))
    print("第 %d 次訓練：準確度：%.16f" %(i + 1, score[1]))